In [14]:
import re
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.wkt import loads

# 신천 교량별 15분 영향권 데이터

In [2]:
#데이터 읽어오기
df = pd.read_csv('D:/jupiter_workspace/intern/dip_intern/sincheon/02population/bridge_polygon.csv')
df = df[['name', 'geometry']]
df.columns = ['bridge', 'geometry']

#geometry 열 polygon type으로 생성
def load_valid(geo):
    try:
        return loads(geo)
    except:
        return np.nan
df.geometry = df.geometry.apply(load_valid)

gdf_sc = gpd.GeoDataFrame(df, geometry='geometry', crs='epsg:4326')
gdf_sc.head()

,bridge,geometry
0,가창,"POLYGON ((128.58400 35.89977, 128.58291 35.900..."
1,경대칠성,"POLYGON ((128.60581 35.86899, 128.60210 35.869..."
2,대봉희망,"POLYGON ((128.61472 35.84350, 128.61354 35.843..."
3,도청경대,"POLYGON ((128.59590 35.88019, 128.59317 35.883..."
4,동신수성,"POLYGON ((128.61729 35.85601, 128.60014 35.859..."


# 트립어드바이저 리뷰 데이터
관광지 위치정보가 누락되어 앞서 정제한 데이터에서 관광지명만 가져와서 추출

In [6]:
#데이터 가져오기
df_trip = pd.read_csv('D:/jupiter_workspace/intern/dip_intern/sincheon/rawdata/31. 대구관광리뷰데이터/트립어드바이저_대구관광_리뷰데이터(utf-8).csv')

#한국어 데이터만 가져오기
df_trip = df_trip[df_trip['LANGUAGE'] == '한국어'].reset_index(drop=True)
df_trip.head()

,Unnamed: 0,NAME,ADDRESS,STAR_AVG,STAR_DIST_DECENDING,REVIEW_TITLE,REVIEW_STAR,REVIEW,VISIT_DATE,WRITE_DATE,LANGUAGE
0,31,신세계백화점 대구점,대구 동구 동부로 149,4.0,"['11', '14', '4', '0', '0']",대구 최적 입지의 백화점,5.0,"비행기를 제외한 고속버스, 시외버스, 기차를 타기 위해선 꼭 들려야 하는 장소.\n...",2020년 6월 • 가족,2020년 6월 27일 작성,한국어
1,35,신세계백화점 대구점,대구 동구 동부로 149,4.0,"['11', '14', '4', '0', '0']",대구 최고의 랜드마크,5.0,신세계 백화점 같은 곳이 없어서 항상 아쉬웠는데 큰 백화점이 들어와서 너무 좋다. ...,2019년 8월 • 커플,2019년 8월 11일 작성,한국어
2,38,신세계백화점 대구점,대구 동구 동부로 149,4.0,"['11', '14', '4', '0', '0']",아쿠아리움있는 백화점,4.0,"주말 아이를 데리고 아쿠아리움을 가거나 기차타러 가거나, 시외로 버스탈 때 꼭 들리...",2019년 3월 • 가족,2019년 3월 15일 작성,한국어
3,41,신세계백화점 대구점,대구 동구 동부로 149,4.0,"['11', '14', '4', '0', '0']",옥상 어린이 놀이터,5.0,신세계 대구점.\n동대구역과 가까이 있어요.\n특히 8층.9층 푸드코트와 어린이 실...,2019년 2월 • 가족,2019년 2월 27일 작성,한국어
4,43,신세계백화점 대구점,대구 동구 동부로 149,4.0,"['11', '14', '4', '0', '0']",먹거리 나들이,5.0,쇼핑하고 점심과 저녁사이 간식\n에그타르트와 멘보샤 사서 바로 옆 폴바셋에서 커피와...,2019년 2월 • 커플,2019년 2월 14일 작성,한국어


In [38]:
#앞서 작업한 new.ipynb에서 scSight.name.unique()로 확인한 관광지명 리스트
L_name = ['공룡공원', '국채보상운동기념공원', '꽃보라동산', '대현어린이공원', '대구 삼성 창조 캠퍼스', '칠성시장',
       '대구제일교회', '건들바위', '방천시장', '달구벌대종', '김광석 다시 그리기 길', '용두산', '들안길공원',
       '건들바위(대구입암)', '아르떼 수성랜드', '수성못', '구암서원 (영남선비문화수련원) 숭현사', '연암공원',
       '침산공원', '대구도시철도', '김광석 길', '수성아쿠아수영장', '김광석스토리하우스', '아이니멀티플렉스', 
       '수성파크랜드']

#L_name에 포함된 곳만 뽑아내기
scSight = df_trip[df_trip['NAME'].isin(L_name)]

scSight.head()

,Unnamed: 0,NAME,ADDRESS,STAR_AVG,STAR_DIST_DECENDING,REVIEW_TITLE,REVIEW_STAR,REVIEW,VISIT_DATE,WRITE_DATE,LANGUAGE
17,85,김광석 길,대구 중구 달구벌대로450길,4.0,"['49', '66', '24', '2', '0']",대구에 간다면 무조건 김광석길,5.0,김광석을 잘 모르는 사람도 리메이크된 노래를 통해서 무조건 한번 쯤은 들어봤을테니 ...,2021년 5월 • 커플,2021년 5월 27일 작성,한국어
19,87,김광석 길,대구 중구 달구벌대로450길,4.0,"['49', '66', '24', '2', '0']",다시는 가지 않을 장소,2.0,"대구에서 김광석 거리가 유명하다고 해서 가봤는데, 김광석 노래 보다는 기계음이 지배...",2021년 4월,2021년 4월 11일 작성,한국어
22,90,김광석 길,대구 중구 달구벌대로450길,4.0,"['49', '66', '24', '2', '0']",토요일 방문,5.0,"12시부터 대구 라디오 방송을 현장에서 진행하고, 관객참여도 같이한다.\n\n한국의...",2019년 6월,2020년 3월 13일 작성,한국어
25,93,김광석 길,대구 중구 달구벌대로450길,4.0,"['49', '66', '24', '2', '0']",기대만큼 그렇게 좋지는 않아요.,3.0,겨울철 저녁시간에 방문해서 그런지 음식점 등만 영업 중이고 조용하네요. 식당에 손님...,2020년 1월 • 친구,2020년 2월 23일 작성,한국어
28,96,김광석 길,대구 중구 달구벌대로450길,4.0,"['49', '66', '24', '2', '0']",김광석 거리를 오랜만에 들렀다.,5.0,김광석 거리를 오랜만에 들렀다. 추억을 더듬어보기에 좋았다. 오전에 들렀드니 거리를...,2020년 1월 • 개인,2020년 1월 26일 작성,한국어


In [39]:
#리뷰 타이틀 + 내용으로 새로운 열 생성
scSight['review'] = scSight['REVIEW_TITLE'] + " " + scSight['REVIEW']

#필요한 열만 가져오기
scSight = scSight[['NAME', 'VISIT_DATE', 'review', 'REVIEW_STAR']]
scSight.columns = ['name', 'date', 'reviews', 'stars']

#date열 보정
scSight.dropna(inplace=True)    #결측치 제거
scSight['date'] = scSight['date'].apply(lambda x : x.split("•")[0])   #동반인 정보 제거
scSight['date'] = scSight['date'].apply(lambda x : re.sub("[ ]$", "", x))   #마지막 띄워쓰기 제거

#df 양식 맞춰주기
scSight['date'] = pd.to_datetime(scSight['date'], format="%Y년 %m월").dt.strftime('%Y-%m')
scSight['stars'] = scSight['stars'].astype(int)
scSight = scSight.reset_index(drop=True)

scSight.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22248\903585730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scSight['review'] = scSight['REVIEW_TITLE'] + " " + scSight['REVIEW']


,name,date,reviews,stars
0,김광석 길,2021-05,대구에 간다면 무조건 김광석길 김광석을 잘 모르는 사람도 리메이크된 노래를 통해서 ...,5
1,김광석 길,2021-04,"다시는 가지 않을 장소 대구에서 김광석 거리가 유명하다고 해서 가봤는데, 김광석 노...",2
2,김광석 길,2019-06,"토요일 방문 12시부터 대구 라디오 방송을 현장에서 진행하고, 관객참여도 같이한다....",5
3,김광석 길,2020-01,기대만큼 그렇게 좋지는 않아요. 겨울철 저녁시간에 방문해서 그런지 음식점 등만 영업...,3
4,김광석 길,2020-01,김광석 거리를 오랜만에 들렀다. 김광석 거리를 오랜만에 들렀다. 추억을 더듬어보기에...,5


In [40]:
scSight.to_csv('31trip_new.csv')